# Wines Points prediction 

Submission Date : 3.6.2023
Task: Predict the wine score given the inputs
Instructions:
 * Use logistic regression as benchmark model
 * Use sklearn pipeliens + cv + grid search with sklearn models (e.g. KNNs, RandomForest, etc.)
 * Compare all models on proper metric (your choice)

For DNN course project:
* Use sklearn pipeliens with tensorflow models (w/wo embeddings, LSTMs, RNNs, Transformers etc.)
* Compare all models on proper metric (your choice)

In [1]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [2]:
import pandas as pd
import cufflinks as cf; cf.go_offline()

In [3]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate, Flatten, TextVectorization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical, plot_model, pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import activations

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


# Embedding

In [131]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.layers import Input, Embedding, LSTM, Concatenate, Dense
from tensorflow.keras.models import Model

# Load your dataset
wine_reviews = pd.read_csv("clean_wine_reviews_data.csv")
summary_df = pd.read_csv("summary_df.csv")

# Preprocessing
wine_reviews = wine_reviews.drop(['Unnamed: 0', 'taster_twitter_handle', 'region_1', 'region_2', 'designation'], axis=1)
wine_reviews['description_length'] = wine_reviews.description.str.len()
wine_reviews = wine_reviews.dropna(subset=['year', 'points'])
wine_reviews['year'] = wine_reviews['year'].astype(int).astype(str)
wine_reviews["description_length"] = wine_reviews["description_length"].astype(np.int32)

# Split data
y = wine_reviews['points']
X = wine_reviews.drop(['points'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=29)

# Preprocess categorical data
categorical_cols = ['country', 'taster_name', 'variety', 'province', 'year']
numeric_cols = ['price']

# Use StandardScaler for numeric columns
numeric_transformer = StandardScaler()

# Use OneHotEncoder for categorical columns
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse=False)

# Preprocess features using ColumnTransformer
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Fit and transform preprocessing on training data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Text Vectorization
max_vocab_length = 10000
max_length = 60

text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length)

text_vectorizer.adapt(X_train['description'])
text_input = Input(shape=(1,), dtype="string")
text_vector = text_vectorizer(text_input)
text_embedding = Embedding(
    input_dim=max_vocab_length,
    output_dim=128,
    embeddings_initializer="uniform",
    input_length=max_length)(text_vector)
text_lstm = LSTM(64)(text_embedding)

# Combine numeric and categorical data
combined_input = Input(shape=(X_train_transformed.shape[1],), dtype="float32")
concatenated = Concatenate()([text_lstm, combined_input])

# Build the model
x = Dense(16, activation='relu')(concatenated)
x = Dense(8, activation='relu')(x)
output = Dense(1, activation="linear")(x)

model = Model(inputs=[text_input, combined_input], outputs=output)

# Compile the model
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mse'])

# Train the model
model.fit([X_train['description'], X_train_transformed], y_train,
          batch_size=64,
          epochs=10,
          validation_split=0.2) 



C:\Users\lior\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Epoch 1/10
1012/1012 [==============================] - 80s 74ms/step - loss: 467.8390 - mse: 467.8390 - val_loss: 6.1965 - val_mse: 6.1965
Epoch 2/10
1012/1012 [==============================] - 137s 135ms/step - loss: 5.9820 - mse: 5.9820 - val_loss: 5.8643 - val_mse: 5.8643
Epoch 3/10
1012/1012 [==============================] - 81s 80ms/step - loss: 5.8069 - mse: 5.8069 - val_loss: 5.8129 - val_mse: 5.8129
Epoch 4/10
1012/1012 [==============================] - 73s 72ms/step - loss: 5.7820 - mse: 5.7820 - val_loss: 5.7795 - val_mse: 5.7795
Epoch 5/10
1012/1012 [==============================] - 93s 92ms/step - loss: 5.7959 - mse: 5.7959 - val_loss: 5.7881 - val_mse: 5.7881
Epoch 6/10
1012/1012 [==============================] - 151s 149ms/step - loss: 5.8024 - mse: 5.8024 - val_loss: 6.0474 - val_mse: 6.0474
Epoch 7/10
1012/1012 [==============================] - 129s 128ms/step - loss: 5.8364 - mse: 5.8364 - val_loss: 6.2460 - val_mse: 6.2460
Epoch 8/10
1012/1012 [================

In [161]:
# Adapt the text vectorizer to the test data
text_description_test = X_test['description'].values
text_vectorizer.adapt(text_description_test)

# Vectorize the test data
text_description_vectorized = text_vectorizer(text_description_test)
text_description_vectorized = tf.reshape(text_description_vectorized, (-1, 1))

# Preprocess numeric and categorical data for test data
X_test_transformed = preprocessor.transform(X_test)

# Convert to float32
X_test_transformed = X_test_transformed.astype(np.float32)

# Create input data for predictions
text_input_test = text_description_vectorized  # Text input
numeric_categorical_input_test = X_test_transformed  # Numeric and categorical input

# Make predictions
predictions = model.predict([text_input_test, numeric_categorical_input_test])

# Print the predictions
print(predictions)


ValueError: Data cardinality is ambiguous:
  x sizes: 1618200, 26970
Make sure all arrays contain the same number of samples.

In [165]:
numeric_categorical_input_test.shape

(26970, 537)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

# Load your dataset
wine_reviews = pd.read_csv("clean_wine_reviews_data.csv")

            
# Preprocessing
wine_reviews = wine_reviews.drop(['Unnamed: 0', 'taster_twitter_handle', 'region_1', 'region_2', 'designation'], axis=1)
wine_reviews['description_length'] = wine_reviews.description.str.len()
wine_reviews = wine_reviews.dropna(subset=['year', 'points'])
wine_reviews['year'] = wine_reviews['year'].astype(int).astype(str)
wine_reviews["description_length"] = wine_reviews["description_length"].astype(np.int32)

# Split data
y = wine_reviews['points']
X = wine_reviews.drop(['points'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=29)

# Define a range of hyperparameters to search over
first_layer_list = [16, 32, 64, 128]
second_layer_list = [8, 16, 32, 64]
learning_rate_list = [0.001, 0.01, 0.1]

best_model = None
best_score = float('inf')

for first_layer in first_layer_list:
    for second_layer in second_layer_list:
        for lrn_rate in learning_rate_list:
            print(f"Training model with {first_layer} layers, {second_layer} units per layer, and learning rate {lrn_rate}")


            # Preprocess categorical data
            categorical_cols = ['country', 'taster_name', 'variety', 'province', 'year']
            numeric_cols = ['price']

            # Use StandardScaler for numeric columns
            numeric_transformer = StandardScaler()

            # Use OneHotEncoder for categorical columns
            categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse=False)

            # Preprocess features using ColumnTransformer
            from sklearn.compose import ColumnTransformer

            preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, numeric_cols),
                    ('cat', categorical_transformer, categorical_cols)
                ])

            # Fit and transform preprocessing on training data
            X_train_transformed = preprocessor.fit_transform(X_train)
            X_test_transformed = preprocessor.transform(X_test)

            # Text Vectorization
            max_vocab_length = 10000
            max_length = 60

            text_vectorizer = tf.keras.layers.TextVectorization(
                max_tokens=max_vocab_length,
                output_mode="int",
                output_sequence_length=max_length)

            text_vectorizer.adapt(X_train['description'])
            text_input = Input(shape=(1,), dtype="string")
            text_vector = text_vectorizer(text_input)
            text_embedding = Embedding(
                input_dim=max_vocab_length,
                output_dim=128,
                embeddings_initializer="uniform",
                input_length=max_length)(text_vector)
            text_lstm = LSTM(64)(text_embedding)

            # Combine numeric and categorical data
            combined_input = Input(shape=(X_train_transformed.shape[1],), dtype="float32")
            concatenated = Concatenate()([text_lstm, combined_input])

            # Build the model
            x = Dense(first_layer, activation='relu')(concatenated)
            x = Dense(second_layer, activation='relu')(x)
            output = Dense(1, activation="linear")(x)
            
    
            # Define the learning rate variable
            learning_rate = tf.Variable(lrn_rate, trainable=False)
            tf.keras.backend.set_value(learning_rate, lrn_rate)

            model = Model(inputs=[text_input, combined_input], outputs=output)

            # Compile the model
            model.compile(loss=tf.keras.losses.MeanSquaredError(),
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mse'])

            # Train the model
            model.fit([X_train['description'], X_train_transformed], y_train,
                      batch_size=64,
                      epochs=10,
                      validation_split=0.2) 


            loss, mse = model.evaluate([X_test['description'], X_test_transformed], y_test, verbose=0)

            # Check if this is the best model so far
            if mse < best_score:
                best_score = mse
                best_model = model
                

print(f"Best Model: {best_params}")
predictions = best_model.predict([X_test['description'], X_test_transformed])


Training model with 16 layers, 8 units per layer, and learning rate 0.001


C:\Users\lior\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Epoch 1/10
1012/1012 [==============================] - 100s 95ms/step - loss: 766.7939 - mse: 766.7939 - val_loss: 6.3922 - val_mse: 6.3922
Epoch 2/10
1012/1012 [==============================] - 99s 97ms/step - loss: 6.0957 - mse: 6.0957 - val_loss: 5.9298 - val_mse: 5.9298
Epoch 3/10
1012/1012 [==============================] - 102s 101ms/step - loss: 5.8472 - mse: 5.8472 - val_loss: 5.8354 - val_mse: 5.8354
Epoch 4/10
1012/1012 [==============================] - 100s 99ms/step - loss: 5.7909 - mse: 5.7909 - val_loss: 5.7853 - val_mse: 5.7853
Epoch 5/10
1012/1012 [==============================] - 98s 97ms/step - loss: 5.7901 - mse: 5.7901 - val_loss: 5.7890 - val_mse: 5.7890
Epoch 6/10
1012/1012 [==============================] - 100s 99ms/step - loss: 5.7869 - mse: 5.7869 - val_loss: 5.8161 - val_mse: 5.8161
Epoch 7/10
1012/1012 [==============================] - 99s 98ms/step - loss: 5.8130 - mse: 5.8130 - val_loss: 5.9673 - val_mse: 5.9673
Epoch 8/10
1012/1012 [=================

C:\Users\lior\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Epoch 1/10
1012/1012 [==============================] - 142s 134ms/step - loss: 806.8312 - mse: 806.8312 - val_loss: 6.5091 - val_mse: 6.5091
Epoch 2/10
1012/1012 [==============================] - 135s 133ms/step - loss: 6.1795 - mse: 6.1795 - val_loss: 5.9797 - val_mse: 5.9797
Epoch 3/10
1012/1012 [==============================] - 135s 133ms/step - loss: 5.8820 - mse: 5.8820 - val_loss: 5.8622 - val_mse: 5.8622
Epoch 4/10
1012/1012 [==============================] - 133s 132ms/step - loss: 5.8005 - mse: 5.8005 - val_loss: 5.7945 - val_mse: 5.7945
Epoch 5/10
1012/1012 [==============================] - 136s 134ms/step - loss: 5.7780 - mse: 5.7780 - val_loss: 5.8245 - val_mse: 5.8245
Epoch 6/10
1012/1012 [==============================] - 134s 133ms/step - loss: 5.7722 - mse: 5.7722 - val_loss: 5.7858 - val_mse: 5.7858
Epoch 7/10
1012/1012 [==============================] - 133s 132ms/step - loss: 5.7916 - mse: 5.7916 - val_loss: 5.7945 - val_mse: 5.7945
Epoch 8/10
1012/1012 [========

...

Epoch 1/10
1012/1012 [==============================] - 107s 102ms/step - loss: 368.7040 - mse: 368.7040 - val_loss: 5.9278 - val_mse: 5.9278
Epoch 2/10
1012/1012 [==============================] - 105s 103ms/step - loss: 5.8402 - mse: 5.8402 - val_loss: 5.8269 - val_mse: 5.8269
Epoch 3/10
1012/1012 [==============================] - 91s 90ms/step - loss: 5.7972 - mse: 5.7972 - val_loss: 5.7791 - val_mse: 5.7791
Epoch 4/10
1012/1012 [==============================] - 82s 81ms/step - loss: 5.8076 - mse: 5.8076 - val_loss: 5.7889 - val_mse: 5.7889
Epoch 5/10
1012/1012 [==============================] - 82s 81ms/step - loss: 5.8365 - mse: 5.8365 - val_loss: 5.7924 - val_mse: 5.7924
Epoch 6/10
1012/1012 [==============================] - 82s 81ms/step - loss: 5.8851 - mse: 5.8851 - val_loss: 5.8112 - val_mse: 5.8112
Epoch 7/10
1012/1012 [==============================] - 82s 81ms/step - loss: 5.8979 - mse: 5.8979 - val_loss: 5.9346 - val_mse: 5.9346
Epoch 8/10
1012/1012 [==================

C:\Users\lior\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Epoch 1/10
1012/1012 [==============================] - 96s 90ms/step - loss: 338.7926 - mse: 338.7926 - val_loss: 5.9332 - val_mse: 5.9332
Epoch 2/10
1012/1012 [==============================] - 92s 91ms/step - loss: 5.8406 - mse: 5.8406 - val_loss: 5.8251 - val_mse: 5.8251
Epoch 3/10
1012/1012 [==============================] - 113s 111ms/step - loss: 5.7946 - mse: 5.7946 - val_loss: 5.7795 - val_mse: 5.7795
Epoch 4/10
1012/1012 [==============================] - 113s 111ms/step - loss: 5.8082 - mse: 5.8082 - val_loss: 5.7932 - val_mse: 5.7932
Epoch 5/10
1012/1012 [==============================] - 118s 116ms/step - loss: 5.8376 - mse: 5.8376 - val_loss: 5.7924 - val_mse: 5.7924
Epoch 6/10
1012/1012 [==============================] - 112s 110ms/step - loss: 5.8856 - mse: 5.8856 - val_loss: 5.8159 - val_mse: 5.8159
Epoch 7/10
1012/1012 [==============================] - 114s 112ms/step - loss: 5.9006 - mse: 5.9006 - val_loss: 5.9359 - val_mse: 5.9359
Epoch 8/10
1012/1012 [============

C:\Users\lior\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Epoch 1/10
1012/1012 [==============================] - 111s 106ms/step - loss: 455.1553 - mse: 455.1553 - val_loss: 5.9707 - val_mse: 5.9707
Epoch 2/10
1012/1012 [==============================] - 112s 110ms/step - loss: 5.8593 - mse: 5.8593 - val_loss: 5.8211 - val_mse: 5.8211
Epoch 3/10
1012/1012 [==============================] - 115s 114ms/step - loss: 5.7924 - mse: 5.7924 - val_loss: 5.7786 - val_mse: 5.7786
Epoch 4/10
1012/1012 [==============================] - 107s 106ms/step - loss: 5.8023 - mse: 5.8023 - val_loss: 5.8134 - val_mse: 5.8134
Epoch 5/10
1012/1012 [==============================] - 109s 108ms/step - loss: 5.8274 - mse: 5.8274 - val_loss: 5.7799 - val_mse: 5.7799
Epoch 6/10
 236/1012 [=====>........................] - ETA: 1:15 - loss: 5.8870 - mse: 5.8870